# Chapter 18: Reinforcement Learning

Bab ini memperkenalkan **Reinforcement Learning (RL)**, salah satu bidang Machine Learning yang paling menarik. Berbeda dengan *supervised* atau *unsupervised learning*, RL berfokus pada bagaimana sebuah **agent** harus mengambil **tindakan** di dalam sebuah **lingkungan** untuk memaksimalkan total **reward** kumulatif.

RL telah mencapai kesuksesan luar biasa dalam berbagai aplikasi, mulai dari mengalahkan juara dunia dalam permainan seperti Go (AlphaGo) hingga mengoptimalkan sistem yang kompleks.

Kita akan membahas:
* Konsep dasar RL: *agent*, *environment*, *rewards*, dan *policy*.
* Dua teknik utama dalam Deep Reinforcement Learning: **Policy Gradients (PG)** dan **Deep Q-Networks (DQNs)**.
* **Markov Decision Processes (MDPs)** sebagai kerangka kerja matematis untuk RL.
* Penggunaan pustaka **OpenAI Gym** untuk lingkungan simulasi.
* Penggunaan pustaka **TF-Agents** untuk membangun sistem RL yang kuat dengan TensorFlow.

## Policy Search

Algoritma yang digunakan *agent* untuk menentukan tindakannya disebut **policy**. *Policy* ini bisa berupa apa saja, dari aturan sederhana hingga jaringan saraf yang kompleks. **Policy Search** adalah pendekatan di mana kita mencari parameter terbaik untuk sebuah *policy* yang akan memaksimalkan *reward*.

Salah satu pendekatan yang paling populer adalah **Policy Gradients (PG)**. Dalam PG, kita menggunakan Gradient Descent untuk men-tweak parameter *policy* ke arah yang akan meningkatkan *reward*. Ini bekerja dengan cara:
1. Menjalankan *policy* beberapa kali dan mengumpulkan hasilnya (*rewards*).
2. Menghitung gradien yang akan membuat tindakan-tindakan yang menghasilkan *reward* tinggi lebih mungkin terjadi.
3. Memperbarui parameter *policy* menggunakan gradien tersebut.

## Introduction to OpenAI Gym

Untuk melatih *agent*, kita memerlukan lingkungan yang berfungsi. **OpenAI Gym** adalah *toolkit* yang menyediakan berbagai lingkungan simulasi (misalnya, game Atari, simulasi fisika) yang menjadi standar untuk penelitian RL.

In [ ]:
import gym

# Membuat lingkungan CartPole
env = gym.make("CartPole-v1")
obs = env.reset()

# Menjalankan satu episode dengan policy acak
for step in range(200):
    action = env.action_space.sample() # memilih tindakan acak
    obs, reward, done, info = env.step(action)
    if done:
        obs = env.reset()
        break

env.close()

## Deep Q-Networks (DQNs)

Alih-alih secara langsung mengoptimalkan *policy*, pendekatan lain adalah dengan mempelajari nilai dari setiap tindakan. **Q-Learning** adalah algoritma yang belajar memperkirakan nilai optimal dari setiap pasangan (state, action), yang disebut **Q-Value**.

**Deep Q-Network (DQN)** adalah implementasi dari Q-Learning menggunakan jaringan saraf dalam untuk mengaproksimasi Q-Values. Ini sangat efektif untuk masalah dengan ruang *state* yang sangat besar (misalnya, input dari piksel layar game).

Teknik kunci dalam DQN adalah penggunaan **replay buffer** (atau *replay memory*), di mana pengalaman (*state, action, reward, next_state*) disimpan dan kemudian di-sampel secara acak untuk melatih jaringan. Ini membantu mengurangi korelasi antar pengalaman dan menstabilkan pelatihan.

In [ ]:
from tensorflow import keras
import numpy as np

# Konseptual: Membangun DQN untuk CartPole
input_shape = [4] # obs: [cart_pos, cart_vel, pole_angle, pole_vel]
n_outputs = 2 # actions: [left, right]

model_dqn = keras.models.Sequential([
    keras.layers.Dense(32, activation="elu", input_shape=input_shape),
    keras.layers.Dense(32, activation="elu"),
    keras.layers.Dense(n_outputs)
])

## The TF-Agents Library

Mengimplementasikan algoritma RL dari awal bisa sangat rumit. **TF-Agents** adalah pustaka dari Google yang menyediakan komponen-komponen yang dapat digunakan kembali dan teruji untuk membangun sistem RL yang kuat dan dapat diskalakan.

TF-Agents menyediakan lingkungan, agen (yang mengimplementasikan algoritma seperti DQN dan PPO), *replay buffer*, metrik, dan *driver* untuk menjalankan loop interaksi antara *agent* dan *environment*.

### Visualisasi Pelatihan RL

Metrik kinerja yang paling penting dalam RL adalah *reward* total yang didapat per episode. Memplot metrik ini dari waktu ke waktu menunjukkan apakah *agent* kita benar-benar belajar.

In [ ]:
import matplotlib.pyplot as plt

# Data reward dummy dari proses pelatihan (konseptual)
episodes = np.arange(1, 601)
# Reward awal rendah dan tidak stabil, kemudian meningkat tajam
rewards_part1 = np.random.normal(loc=20, scale=8, size=300)
rewards_part2 = np.random.normal(loc=190, scale=15, size=200).clip(min=100, max=200)
rewards_part3 = np.random.normal(loc=50, scale=10, size=100) # Contoh catastrophic forgetting
rewards = np.concatenate([rewards_part1, rewards_part2, rewards_part3])
rewards = np.clip(rewards, 0, 200)

# Menghitung rata-rata bergerak untuk memperhalus kurva
def moving_average(data, window_size):
    return np.convolve(data, np.ones(window_size), 'valid') / window_size

smoothed_rewards = moving_average(rewards, 30)

plt.figure(figsize=(10, 6))
plt.plot(episodes, rewards, label='Reward per Episode', alpha=0.3)
plt.plot(episodes[len(episodes)-len(smoothed_rewards):], smoothed_rewards, 'r-', label='Rata-rata Bergerak (30 Episode)')
plt.xlabel("Episode")
plt.ylabel("Total Reward")
plt.title("Kurva Pembelajaran Agent RL")
plt.legend()
plt.grid(True)
plt.show()